In [1]:
from transformers import AutoModelForCausalLM

#model_id = "meta-llama/Llama-2-7b-hf"
model_id = "facebook/opt-1.3b"
model = AutoModelForCausalLM.from_pretrained(model_id)

/home/jwjeong/anaconda3/envs/quant/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jwjeong/anaconda3/envs/quant/lib/python3.8/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
from transformers.pytorch_utils import Conv1D
import torch
import torch.nn as nn

def get_weight_minmax_gpt2(model):
    weight_min = {}
    weight_max = {}
    for name, module in model.named_moWdules():
        if (isinstance(module, Conv1D)) | isinstance(module, nn.Linear):
            if name.split('.')[-1] in ['c_fc', 'c_proj', 'c_attn']:
                if name.split('.')[-2] in ['attn', 'mlp']:
                    comp_name = '.'.join(name.split('.')[-2:])
                    if  comp_name in weight_min:
                        weight_min[comp_name].append(torch.min(module.weight).item())
                        weight_max[comp_name].append(torch.max(module.weight).item())
                    else:
                        weight_min[comp_name] = []
                        weight_max[comp_name] = []
                        weight_min[comp_name].append(torch.min(module.weight).item())
                        weight_max[comp_name].append(torch.max(module.weight).item())
    
    return weight_min, weight_max

def get_weight_minmax_opt(model):
    weight_min = {}
    weight_max = {}
    for name, module in model.named_modules():
        if (isinstance(module, Conv1D)) | isinstance(module, nn.Linear):
            if name.split('.')[-1] in ['q_proj', 'k_proj', 'v_proj', 'out_proj', 'fc1', 'fc2']:
                comp_name = '.'.join(name.split('.')[-1:])
                if  comp_name in weight_min:
                    weight_min[comp_name].append(torch.min(module.weight).item())
                    weight_max[comp_name].append(torch.max(module.weight).item())
                else:
                    weight_min[comp_name] = []
                    weight_max[comp_name] = []
                    weight_min[comp_name].append(torch.min(module.weight).item())
                    weight_max[comp_name].append(torch.max(module.weight).item())
    
    return weight_min, weight_max

def get_weight_minmax_llama(model):
    weight_min = {}
    weight_max = {}
    for name, module in model.named_modules():
        if (isinstance(module, Conv1D)) | isinstance(module, nn.Linear):
            if name.split('.')[-1] in ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'up_proj', 'down_proj', 'gate_proj']:
                comp_name = '.'.join(name.split('.')[-1:])
                if  comp_name in weight_min:
                    weight_min[comp_name].append(torch.min(module.weight).item())
                    weight_max[comp_name].append(torch.max(module.weight).item())
                else:
                    weight_min[comp_name] = []
                    weight_max[comp_name] = []
                    weight_min[comp_name].append(torch.min(module.weight).item())
                    weight_max[comp_name].append(torch.max(module.weight).item())
    
    return weight_min, weight_max

In [15]:
from quant import quant, dequant
bits = 16
gs = 128
scale, zero, qs = quant(bits, gs, model)
q_x = dequant(scale, zero, qs, gs, bits)
for key in q_x.keys():
    if key.split('.')[-1] != 'lm_head':
        weight = key+'.weight'
        model.state_dict()[weight][:] = q_x[key]

In [16]:
min, max = get_weight_minmax_opt(model)

In [17]:
for key in min.keys():
    print(key, min[key])

k_proj [-0.24174661934375763, -0.27865371108055115, -0.27822643518447876, -0.2794552445411682, -0.28324615955352783, -0.29386502504348755, -0.3181660771369934, -0.29851433634757996, -0.2978433072566986, -0.29903721809387207, -0.28878435492515564, -0.29673653841018677, -0.2962765395641327, -0.299712598323822, -0.2977583408355713, -0.28110450506210327, -0.28653380274772644, -0.28155767917633057, -0.2879585921764374, -0.2852679491043091, -0.2938303053379059, -0.2783399820327759, -0.2747690975666046, -0.27728962898254395]
v_proj [-0.13668617606163025, -0.18865646421909332, -0.14202116429805756, -0.1341332644224167, -0.11230236291885376, -0.11180998384952545, -0.10690794885158539, -0.16197851300239563, -0.14110782742500305, -0.11453115940093994, -0.12665385007858276, -0.13032442331314087, -0.1416580229997635, -0.11882098764181137, -0.1360282003879547, -0.1497844010591507, -0.12516529858112335, -0.14367319643497467, -0.1507648229598999, -0.16286520659923553, -0.1891021579504013, -0.170041814

In [18]:
for key in max.keys():
    print(key, max[key])

k_proj [0.2100357562303543, 0.2476140260696411, 0.2359100580215454, 0.2441413849592209, 0.24294601380825043, 0.2544305622577667, 0.27317431569099426, 0.2588602602481842, 0.25662460923194885, 0.26618748903274536, 0.255293071269989, 0.2631424069404602, 0.26078569889068604, 0.26001396775245667, 0.26080653071403503, 0.24541430175304413, 0.25928768515586853, 0.2481663078069687, 0.24270963668823242, 0.2424391359090805, 0.2517335116863251, 0.24165232479572296, 0.24728302657604218, 0.2423408329486847]
v_proj [0.0793766975402832, 0.1335023194551468, 0.1391310840845108, 0.1291218250989914, 0.13099807500839233, 0.09936639666557312, 0.08989805728197098, 0.1453159600496292, 0.12123468518257141, 0.10891036689281464, 0.09447711706161499, 0.10221104323863983, 0.13590817153453827, 0.1189442127943039, 0.12522746622562408, 0.13419455289840698, 0.11442945152521133, 0.12514424324035645, 0.12424957752227783, 0.13308241963386536, 0.16024285554885864, 0.14878658950328827, 0.1554727703332901, 0.145667761564254